This KalmanFilter class implements a standard Kalman Filter algorithm for the lunar case, which is used for estimating the state of a dynamic system from a series of noisy measurements. The class is initialized with system matrices such as the state transition matrix (A), observation matrix (H), and noise covariance matrices (Q and R), among others.

It includes methods for both the prediction and update steps of the Kalman Filter, allowing it to iteratively refine state estimates as new measurements become available. Additionally, the class features a msmt_sharing method that simulates the sharing of measurement data between different receivers in a network, updating the state estimates based on the biases of neighboring nodes.

In [ ]:
class KalmanFilter(object):
  # Initialize the Kalman Filter with parameters
  def __init__(self, A = None, B = None, H = None, Q = None, R = None, P = None, x0 = None):
    if(A is None or H is None):
      raise ValueError("Set proper system dynamics.")

    # number of state variables
    self.n = A.shape[1]

    # number of measurement variables

    # assign the system matrices, using default values
    self.A = A
    self.H = H
    self.B = np.zeros((1,1)) if B is None else B

    # covariance is often a diagonal matrix since variables are independent
    self.Q = np.eye(self.n) if Q is None else Q # process noise covariance matrix
    self.R = np.eye(self.n) if R is None else R # measurement noise covariance matrix
    self.P = np.eye(self.n) if P is None else P # the initial error estimate covariance matrix
    self.x = np.zeros((self.n, 1)) if x0 is None else x0 # the initial state estimate vector

  # Prediction step of the Kalman filter
  def predict(self, u = None):
    if u is None: u = np.zeros((self.B.shape[1],1))

    self.x = (self.A @ self.x) + (self.B @ u) # predict the next state estimate
    self.P = ((self.A @ self.P) @ self.A.T) + self.Q # predict the next estimate error covariance
    return self.x

  # Update step of the Kalman Filter
  def update(self, z, num_msmts):
    S = self.R(num_msmts) + (self.H(num_msmts) @ (self.P @ self.H(num_msmts).T)) # calculate the measurement residual covariance
    K = sp.linalg.cho_solve(sp.linalg.cho_factor(S.T),(self.H(num_msmts) @ self.P.T)).T # Kalman gain using a Cholesky solver
    y = z - (self.H(num_msmts) @ self.x) # calculate the measurement residual
    self.x = self.x + (K @ y) # update the state estimate

    # update the estimate error covariance
    I = np.eye(self.n)
    self.P = (((I - (K @ self.H(num_msmts))) @ self.P) @ ((I - (K @ self.H(num_msmts))).T))+(K @ (self.R(num_msmts) @ K.T))
    return K

  def msmt_sharing(self, true_global, true_bias):
    new_P = np.copy(self.P)
    psi_x = np.copy(self.x)
    for reciever in range(self.n//2):
      H_ms = np.array([[1 if i == 2*reciever else 0 for i in range(self.n)]])
      for sender in range(self.n//2):
        if reciever != sender:
          #Covariance is variance of neighbors bias
          stn_P = np.array([[10*new_P[2*sender][2*sender]]])


          S = stn_P + (H_ms @ (self.P @ H_ms.T)) # calculate the measurement residual covariance
          K = sp.linalg.cho_solve(sp.linalg.cho_factor(S.T),(H_ms @ new_P.T)).T # Kalman gain using a Cholesky solver

          reciever_local_time = (true_global+true_bias[reciever])
          sender_local_time = (true_global + true_bias[sender])
          sender_global_est = sender_local_time - (1/c)*psi_x[2*sender]
          bias_est = reciever_local_time - sender_global_est
          z = c*bias_est


          y = z - (H_ms @ psi_x) # calculate the measurement residual
          psi_x = psi_x + (K @ y) # update the state estimate

          # update the estimate error covariance
          I = np.eye(self.n)
          new_P = (((I - (K @ H_ms)) @ new_P) @ ((I - (K @ H_ms)).T))+(K @ (stn_P @ K.T))

      self.P = new_P
      self.x = psi_x
